#### Devido a organização das pastas, uso as bibliotecas "os" e "sys" para definidir o local como sendo na pasta anterior, a "Controle_jubilee"

In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

#### Em seguida, faço a importação os objetos e funções dos scripts Python. Recomendo que acesse esses arquivo .py para conferir melhor as 

In [2]:
from jubilee_controller import JubileeMotionController
import matplotlib.pyplot as plt
import time
import numpy as np 
from pynput import keyboard
import cv2
from datetime import datetime
from camera_controller import camera_tool
from email_sender import send_email
from image_processing import detect_circle

In [3]:
jubilee = JubileeMotionController()

In [4]:
jubilee.reset()

Reconectando...


In [5]:
jubilee.home_all(mesh_mode_z=False)

In [6]:
jubilee.move_xyz_absolute(z=60)

In [7]:
jubilee.controlar_jubilee()

AttributeError: 'JubileeMotionController' object has no attribute 'controlar_jubilee'

In [10]:
camera = camera_tool(jubilee,(302,10))

In [12]:
camera.uninstall()

In [8]:
for i in range(1):
    camera.install()
    camera.uninstall()
    time.sleep(5)

In [8]:
from gripper_controller import Gripper

In [9]:
gri = Gripper(jubilee)
camera = camera_tool(jubilee)

In [11]:
for _ in range(0,1):
    gri.install()
    time.sleep(1)
    gri.uninstall()

In [13]:
for _ in range(0,2):
    camera.install()
    time.sleep(1)
    camera.uninstall()
    time.sleep(1)
    gri.install()
    time.sleep(1)
    gri.uninstall()

In [51]:
jubilee.move_xyz_absolute(z=220)

In [35]:
gri.install()

In [99]:


class Pipette:
    def __init__(self,machine,parking_position_xy=(138,18)):

        self.parking_position_x,self.parking_position_y = parking_position_xy
        self.machine = machine
        self.move_velocity = 10000

        self.machine.gcode("M208 Z100:300")
        self.machine.gcode('M98 P"/sys/homev.g"')
        if self.machine.position[2]<100:
            self.machine.move_xyz_absolute(z=100)
        
        self.liquid_ul=0

    def install(self):

        self.machine.protect_tools(on=False)
        
        self.machine.move_xyz_absolute(y=220, velocity=self.move_velocity)
        self.machine.move_xyz_absolute(x=self.parking_position_x, velocity=self.move_velocity)
        self.machine.gcode("G0 U70")
        self.machine.move_xyz_absolute(y=self.parking_position_y, velocity=self.move_velocity)
        self.machine.gcode("G0 U0")
        self.machine.move_xyz_absolute(y=70, velocity=self.move_velocity)

        if self.machine.mode_protect_tools:
            self.machine.protect_tools(on=False)

    def uninstall(self):

        self.machine.protect_tools(on=False)

        self.machine.move_xyz_absolute(y=90, velocity=self.move_velocity)
        self.machine.move_xyz_absolute(x=self.parking_position_x, velocity=self.move_velocity)
        self.machine.move_xyz_absolute(y=self.parking_position_y, velocity=self.move_velocity)
        self.machine.gcode("G0 U70")
        self.machine.move_xyz_absolute(y=70, velocity=self.move_velocity)
        self.machine.gcode("G0 U0")

        if self.machine.mode_protect_tools:
            self.machine.protect_tools(on=False)

    
    def press(self,ul):
        if ul>1200:
            print("Não tente pipetar mais que 1200ul")
            pass

        
        
        if self.liquid_ul > 0:
            self.machine.gcode("G0 V350")
            self.liquid_ul = 0
        
        next_positon = ul/4
        self.machine.gcode(f"G0 V{next_positon}")
    

    def aspirate(self):
        positions = self.machine.gcode("M114")
        valor_v = float(positions.split("V:")[1].split("E:")[0].strip())
        if valor_v==0:
            print("Use o método 'press' antes")
            pass
        
        self.liquid_ul = valor_v*4
        self.machine.gcode("G0 V0")
    
    def dispense(self):
        self.machine.gcode("G0 V400")
        self.liquid_ul = 0

    def drop(self):
            self.machine.gcode("G0 V450")
            self.liquid_ul = 0
            self.machine.gcode("G0 V0")


In [10]:
jubilee.protect_tools(True)

In [100]:
pipeta = Pipette(jubilee)

In [79]:
pipeta.install()

In [54]:
jubilee.move_xyz_absolute(y=200)

In [62]:
pipeta.drop()

In [63]:
jubilee.move_xyz_absolute(x=188,y=335)
jubilee.move_xyz_absolute(z=160,velocity=4000)
jubilee.move_xyz_absolute(z=250,velocity=4000)

In [92]:
pipeta.press(100)

In [95]:
positions = jubilee.gcode("M114")
valor_v = float(positions.split("V:")[1].split("E:")[0].strip())
valor_v

0.0

In [89]:
pipeta.liquid_ul

0

In [103]:
for i in range(9):  
    jubilee.move_xyz_absolute(x=88,y=310,velocity=15000)
    pipeta.press(1000)
    jubilee.move_xyz_absolute(z=165,velocity=15000)
    pipeta.aspirate()
    time.sleep(1)
    jubilee.move_xyz_absolute(z=285,velocity=15000)
    jubilee.move_xyz_absolute(x=88,y=165,velocity=15000)
    pipeta.dispense()

In [104]:
pipeta.drop()

In [105]:
pipeta.uninstall()